In [ ]:
import subprocess
import glob
from tqdm import tqdm
from tile_operator.operate import TileOperate, file_to_bounds

In [ ]:
# https://geojson.io/ でgeojsonを作成
file_path = "./wajima.geojson"
bbox = file_to_bounds(file_path).bounds()
print(bbox)

to = TileOperate(
    bbox=bbox,
    zoom_level=18,
)

to.set_tile_list()
tile_list = to.get_tile_list()
print(len(tile_list))

# 地図タイルを取得し、一枚の画像を作成

全国最新写真（シームレス）
URL：https://cyberjapandata.gsi.go.jp/xyz/seamlessphoto/{z}/{x}/{y}.jpg

令和6年能登半島地震 輪島東地区　正射画像（2024年1月2日撮影）
URL：https://cyberjapandata.gsi.go.jp/xyz/20240102noto_wazimahigashi_0102do/{z}/{x}/{y}.png

In [ ]:
# 災害前と災害後のタイル
tile_list = [("seamlessphoto", "jpg")
             , ("20240102noto_wazimanaka_0102do", "png")]

In [ ]:
for tile_name, tile_format in tile_list:
    tile_url = f"https://cyberjapandata.gsi.go.jp/xyz/{tile_name}" + "/{z}/{x}/{y}." + tile_format
    print(tile_url)

    # 地図タイルを取得
    for tile in tqdm(to.tile_list):        
        to.download_tile(tile_url, tile[0], tile[1], output=f"./{tile_name}")
        tile_path = f"./{tile_name}/18/{tile[0]}/{tile[1]}.{tile_format}"
        to.tile_to_geotiff(tile_path) # 地図タイルをgeotiffへ変換
    
    files = []
    for y_name in tqdm(glob.glob(f"./{tile_name}/18/*")):
        files = files + glob.glob(f"{y_name}/*.tif")

    # gdal_mergeで各geotiff画像を一枚のgeotiff画像にする
    command = ["gdal_merge.py", "-o", f"./{tile_name}_18.tif", "-co", "BIGTIFF=YES"] + files
    print(command)
    subprocess.run(command)